<a href="https://colab.research.google.com/github/harshapilli123/fmmllabs/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [ ]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [ ]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [ ]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [ ]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [ ]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?

The size of the validation set can have an impact on the accuracy of your model's validation performance. Here's how it's affected when you increase or decrease the percentage of the validation set:

**Increasing the Percentage of Validation Set:**

1. **Pros:**
   - More data for validation: With a larger validation set, you have more data available for assessing your model's performance. This can result in a more accurate estimate of how well your model generalizes to unseen data.
   - Better representation: A larger validation set may better represent the distribution of your data, reducing the risk of the validation set being unrepresentative of the test data.

2. **Cons:**
   - Smaller training set: As you allocate more data to the validation set, you have less data available for training. This can potentially lead to the model not learning as effectively, especially if your dataset is limited in size.

**Decreasing the Percentage of Validation Set:**

1. **Pros:**
   - Larger training set: With a smaller validation set, you have more data available for training. This can allow your model to learn better and potentially improve its performance.

2. **Cons:**
   - Less reliable validation: A smaller validation set may provide a less reliable estimate of your model's performance because it's based on fewer data points. It's more susceptible to random variations and might not generalize well to unseen data.
   - Risk of overfitting: A small validation set can make it harder to detect overfitting. Your model may appear to perform well on the validation set, but it might not generalize to new data.

In summary, increasing the percentage of the validation set generally leads to a more reliable estimate of your model's performance but can result in a smaller training set. Decreasing the percentage of the validation set can provide a larger training set but may lead to less reliable validation results. The choice of the validation set size should be a trade-off between having enough data for model training and having a representative and reliable estimate of model performance. Techniques like cross-validation can help mitigate some of these trade-offs by repeatedly shuffling and splitting the data into training and validation sets.
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?

The size of the training and validation sets can indeed affect how well you can predict the accuracy of your model on the test set using the validation set. This is especially important in the context of machine learning model development and evaluation. Here's how it works:

1. **Training Set Size:**
   - **Larger Training Set:** When you have a larger training set, your model has more data to learn from. This can often lead to a better-performing model in terms of training accuracy. However, it might also make the model prone to overfitting if it becomes too complex relative to the training data.
   - **Smaller Training Set:** With a smaller training set, the model may not have sufficient data to capture the underlying patterns in the data. This can result in underfitting, where the model doesn't perform well on both the training and validation sets.

2. **Validation Set Size:**
   - **Larger Validation Set:** A larger validation set can provide a more reliable estimate of your model's performance. It reduces the impact of random variations and provides a better indication of how well your model generalizes to unseen data. However, if the validation set is very large relative to the training set, it might lead to an underestimation of your model's performance because it becomes too similar to the training data.
   - **Smaller Validation Set:** A smaller validation set may lead to a less reliable estimate of your model's performance. It's more susceptible to random fluctuations and might not capture the model's true generalization ability accurately. However, it can be useful when you have limited data, and you need a larger training set.

3. **Prediction of Test Set Accuracy:**
   - The validation set is used to estimate how well your model is likely to perform on unseen data, such as the test set. If your validation set is representative of the test set and has a sufficient size, it can provide a reasonably accurate prediction of the model's performance on the test set.
   - However, if your validation set is too small or not representative, it might provide an inaccurate estimate of the test set performance. In such cases, the model's performance on the test set could be better or worse than what the validation set suggests.

In practice, it's essential to strike a balance between the size of the training and validation sets, taking into consideration the total amount of data available, the complexity of the model, and the need for reliable performance estimates. Techniques like cross-validation can help mitigate some of the issues associated with the size of the validation set by repeatedly splitting the data and averaging the results.
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

The percentage of data to reserve for the validation set depends on various factors, including the total size of your dataset, the complexity of your model, and the reliability of the validation estimate you desire. There's no one-size-fits-all answer, but here are some guidelines to help you strike a balance:

1. **80/20 Rule:** A common starting point is to split your data into approximately 80% for training and 20% for validation. This can work well for moderate-sized datasets and models with reasonable complexity.

2. **70/30 Rule:** In cases where you have a limited amount of data, you might consider using a 70/30 or even 60/40 split for training and validation. This allows you to allocate more data for training to help the model learn better, even though your validation estimate might have slightly higher variability.

3. **Cross-Validation:** If you have a small dataset and want a more robust estimate of your model's performance, consider using cross-validation techniques such as k-fold cross-validation or stratified k-fold cross-validation. These methods repeatedly split the data into training and validation subsets, helping you to average out the variations in validation performance.

4. **Data Size Matters:** For very large datasets, you can allocate a smaller percentage to the validation set (e.g., 10%) because you still have a substantial amount of data for both training and validation. In such cases, the size of your validation set can still be adequate for reliable estimates.

5. **Domain Knowledge:** Consider domain-specific knowledge. Some domains may require larger validation sets due to inherent data variability or uncertainty.

6. **Iterative Experimentation:** It's often a good practice to experiment with different validation set sizes and observe how they affect your model's performance. You can then choose the size that provides a good trade-off between training data and reliable validation estimates.

In summary, there is no fixed percentage that works for all situations. The choice of the validation set size should be influenced by the specific characteristics of your dataset and the goals of your machine learning project. You may need to adjust the split percentage and use techniques like cross-validation to find the right balance between training and validation.
Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [ ]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [ ]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?

Yes, averaging the validation accuracy across multiple splits of your dataset, typically achieved through techniques like cross-validation, can indeed give more consistent and reliable results. Here's why:

1. **Reduced Variance:** By using multiple splits of your data, you obtain multiple estimates of your model's performance. These estimates tend to have different random variations due to the different data subsets used for validation in each split. Averaging these estimates helps smooth out these variations, resulting in a more stable and consistent performance metric.

2. **Robustness to Data Variability:** Real-world datasets can have inherent variability, and a single random split may lead to a validation set that doesn't fully represent the overall data distribution. Multiple splits ensure that your model's performance is evaluated on various data samples, reducing the risk of obtaining misleading results due to an unrepresentative split.

3. **Better Generalization Assessment:** Averaging validation accuracy over multiple splits provides a more reliable estimate of how well your model is likely to generalize to unseen data. It offers a more comprehensive view of your model's expected performance on a broader range of data scenarios.

4. **Mitigating Overfitting/Underfitting:** If you use a single validation split, it's possible that your model could perform exceptionally well or poorly just by chance. This can lead to overfitting or underfitting assessments that are not indicative of the model's true capability. Averaging over multiple splits helps reduce this risk.

5. **Hyperparameter Tuning and Model Selection:** When experimenting with different model architectures or hyperparameters, averaging validation accuracy over multiple splits becomes crucial. It provides a more reliable basis for choosing the best model or hyperparameter settings.

In summary, averaging validation accuracy across multiple splits, as accomplished through cross-validation techniques, is a standard practice in machine learning. It enhances the robustness and reliability of your model evaluation and helps you make more informed decisions about model performance, hyperparameters, and generalization ability. It is particularly valuable when working with limited data or when striving for a thorough understanding of your model's behavior.
2. Does it give more accurate estimate of test accuracy?

Averaging validation accuracy across multiple splits, such as through cross-validation, provides a more reliable and accurate estimate of how well your model is likely to perform on unseen data, which is often represented by the test set. Here's why it leads to a more accurate estimate of test accuracy:

1. **Reduced Variance:** By performing cross-validation and averaging the results, you reduce the impact of random variations that can occur when you have a single train-test split. These variations can lead to overly optimistic or pessimistic estimates of model performance. Averaging mitigates this variability, resulting in a more stable and accurate estimate.

2. **Comprehensive Evaluation:** Cross-validation assesses your model's performance on different subsets of the data, ensuring that it's tested on a variety of data points. This provides a more comprehensive evaluation of how well your model generalizes to different scenarios within your dataset, making the estimate more accurate.

3. **Better Model Selection:** When comparing multiple models or tuning hyperparameters, cross-validation allows you to make more informed decisions. Averaging validation accuracy helps you select the model or hyperparameter settings that are more likely to perform well on unseen data (represented by the test set).

4. **Reduced Risk of Data Split Bias:** In a single train-test split, there's a risk that the specific split may not be representative of the entire dataset. Cross-validation repeatedly reshuffles the data and ensures that each data point participates in both training and validation, reducing the risk of biased splits.

5. **Accurate Assessment of Model Behavior:** Cross-validation provides a more accurate assessment of your model's behavior in terms of its ability to generalize to new, unseen data. It helps you avoid making overly optimistic assumptions about your model's performance based on a single random split.

While cross-validation provides a more accurate estimate of test accuracy, it's important to note that the test set remains a critical component of model evaluation. The test set represents truly unseen data, and its accuracy provides the final confirmation of your model's generalization ability. However, cross-validation serves as a valuable tool for making informed decisions during model development and hyperparameter tuning, increasing your confidence in how your model is likely to perform on the test set.
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?

In the context of cross-validation, the number of iterations, often referred to as "folds," can affect the accuracy and reliability of the estimate of your model's performance. Generally, increasing the number of iterations (folds) can lead to a better estimate, but there are trade-offs to consider:

**More Iterations (Higher k):**

Pros:
1. **More Robust Estimate:** With more iterations/folds, you obtain multiple estimates of your model's performance, each based on a different subset of the data. Averaging these estimates helps create a more robust and stable performance metric.

2. **Reduced Variability:** Additional folds help reduce the variability of the estimate, as you're evaluating your model on a larger number of data subsets. This can result in a more accurate and less noisy estimate.

3. **Better for Smaller Datasets:** When working with smaller datasets, more iterations can help you make the most of the available data for both training and validation.

Cons:
1. **Increased Computational Cost:** As the number of iterations increases, the computational cost also goes up. Cross-validation with a large number of folds may require significantly more computation time.

2. **Smaller Validation Sets:** With more folds, each validation set becomes smaller, potentially leading to less reliable validation estimates if your dataset is already limited in size.

**Fewer Iterations (Lower k):**

Pros:
1. **Faster Computation:** Cross-validation with fewer iterations is computationally less demanding, making it faster, especially when working with large datasets.

2. **Larger Validation Sets:** A smaller number of folds means larger validation sets, which can provide more reliable estimates when working with limited data.

Cons:
1. **Higher Variability:** With fewer folds, the estimate of your model's performance can be more variable, as it's based on fewer data subsets. This can result in less confidence in the estimate.

2. **Risk of Biased Splits:** A small number of iterations may increase the risk of biased splits, where specific subsets of data might not be representative of the overall dataset.

In practice, the choice of the number of iterations (k) in cross-validation depends on factors like the size of your dataset, the computational resources available, and the desired level of reliability in your performance estimate. Common values for k include 5-fold and 10-fold cross-validation, which strike a balance between computational efficiency and reliable estimates. You can experiment with different values of k to determine what works best for your specific machine learning project.
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?
Increasing the number of iterations (folds) in cross-validation can help mitigate some of the limitations of having a very small training dataset or validation dataset to some extent. However, it's important to understand the trade-offs involved:

**Dealing with a Very Small Training Dataset:**

- **Increasing Iterations (Higher k):** If you have a very small training dataset, increasing the number of iterations (higher k) can be beneficial to make the most of the limited training data. With more folds, you ensure that your model is trained and evaluated on different subsets of the data, which can help improve its generalization.

- **Trade-offs:** While increasing iterations can help, there's a limit to how much it can compensate for a tiny training dataset. If your training dataset is extremely small, your model may still struggle to capture complex patterns, even with cross-validation.

**Dealing with a Very Small Validation Dataset:**

- **Increasing Iterations (Higher k):** More iterations can help alleviate the issue of having a very small validation dataset. It reduces the variability in the estimate of your model's performance because each fold contributes a validation estimate. This can make your validation metric more reliable.

- **Trade-offs:** Increasing the number of iterations cannot create more data for the validation set. If your validation dataset is exceptionally small, even with cross-validation, each fold's validation set will remain small. While averaging over multiple folds reduces variability, it doesn't change the fundamental limitation of having a limited amount of validation data.

In summary, increasing the number of iterations in cross-validation is a valuable strategy to make the most of limited data, whether it's in the training or validation dataset. It helps reduce variability and provides more robust estimates. However, it cannot fully compensate for the challenges posed by extremely small datasets. The choice of the number of iterations (k) should consider the available data and the balance between computation time and reliable estimates. If both the training and validation datasets are exceedingly small, it's important to collect more data if possible, as this is often the most effective way to improve model performance.